# 第10章 設定オブジェクトと設定ファイル(settings.py)

## 10.1 概要
- 設定オブジェクト：`django,conf.settings`
- 設定ファイル：プロジェクト作成時に生成される`settings.py`

### 設定オブジェクト
- Djangoの起動時にインスタンス化されるシングルトンなグローバルオブジェククト
- Djangoのデフォルト設定ファイル（`django.conf.global__settings.py`）を読み込んだ後、プロジェクト作成時に生成される設定ファイル（`settings.py`）を読み込んで設定を上書きする

#### `django/conf/\_\_init\_\_.py`
~~~python
settings = LazySettings()
~~~

### 設定ファイル
- プロジェクト作成時に`settigns.py`として生成される
- Djangoのデフォルト設定とは異なるプロジェクト固有の設定値や、自作のアプリケーションで利用する独自の変数を記述する

### 設定値の参照
~~~python
from django.conf import settings
print(settings.LOGIN_URL)
~~~

## 10.2 インストールするアプリケーション一覧

### `INSTALLED_APPS`
- インストールするアプリケーションをリスト形式で列挙する

#### 自作アプリケーションを追加する
- 各アプリケーションディレクトリ直下の`apps.py`に書かれた`AppConfig`クラスのサブクラスを追加する
~~~python
INSTALLED_APPS = [
    ...（略）...
    'accounts.apps.AccountsConfig',
    'shop.apps.ShopConfig',
]
~~~

##### アプリケーション作成時の`apps.py`の例（`accounts/apps.py`）
~~~python
from django.apps import AppConfig

class AccountsConfig(AppConfig):
    name = 'accounts'
~~~

#### 読み込み順
- 上に書いたほうが優先される
- プロジェクト作成時にデフォルトで設定されている`INSTALLED_APPS`のリストの下の追記する形で記述すれば良い

## 10.3 デバッグ設定

### `DEBUG`
- 開発モードと本番モードを切り替えられるようにする設定
- 開発時は`True`にしておくことで、エラー発生時に画面にデバッグ情報が出力されるなどの開発に便利な機能が提供される
- `DEBUG`が`True`になっていないと利用できない機能もある
  - `django-debug-toolbar`
  - SQL文のロギング
- セキュリティ面から、本番稼働時には`False`に設定しておく

## 10.4 静的ファイル関連の設定

### 静的ファイル
- CSSファイルやJavascriptファイル、画像ファイルなど、リクエストに応じて中身を変更せずにそのまま配信可能なファイル

### 静的ファイルとサーバ
- 単に静的ファイルを返すだけの処理をアプリケーションサーバで捌くのは無駄が多い
  - アプリケーションサーバの前段に「リバースプロキシ」と呼ばれるサーバを配置し、静的ファイルを返すだけの処理はリバースプロキシが担当する
    - リバースプロキシの代表例：Ngnix
- セキュリティなどの観点から、「静的ファイルの配信元」とプロジェクトで静的ファイルをバージョン管理する際の「プロジェクト内の置き場所」を別々にするケースが多い

### Djangoでの静的ファイル配信設定

| 変数 | 説明 |
| :-- | :-- |
| STATIC_URL | 静的ファイル配信用のディレクトリで、URLの一部になる。<br />設定値はデフォルトの「/static/」のままでよい |
| STATICFILES_DIRS | アプリケーションに紐付かない静的ファイルの置き場所 |
| STATIC_ROOT | 静的ファイルの配信元。<br />`collectstatic`コマンドで静的ファイルを集約する際のコピー先でもある。<br />`STATICFILES_DIRS`とは別のディレクトリを指定する必要がある |

#### 静的ファイル関連の設定例（`config/settings.py`）
~~~python
STATIC_URL = '/static/'
STATICFILES_DIRS = [os.path.join(BASE_DIR, 'static')]
STATIC_ROOT = '/var/www/{}/static'.format(PROJECT_NAME)
~~~

#### BASE_DIR, PROJECT_NAMEの設定
以下のような変数を`settings.py`内で設定しておくと何かと便利
- BASE_DIR：プロジェクトを配置したベースディレクトリ
- PROJECT_NAME：プロジェクト名

~~~python
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
PROJECT_NAME = os.path.basename(BASE_DIR)
~~~

#### `STATIC_ROOT`
- `DEBUG`が`False`の場合に必要となる設定（=本番環境で動作させることを想定した設定）
- リバースプロキシを使うなどして配信する場合の配信元

#### `collectstatic`コマンド
- `DEBUG`が`False`の場合に、必要な状態
  - リバースプロキシ側では`STATIC_ROOT`配下の静的ファイルを`http(s)://<ホスト名>/<STATIC_URLの値>/...`のURLリクエストで参照できるように設定する必要がある
  - Django側では、`STATIC_ROOT`配下に静的ファイルを集約する必要がある
- 静的ファイル集約のための管理コマンドが`collectstatic`コマンド
~~~
python3 manage.py collectstatic
~~~

### テンプレートで静的ファイルの配信用URLを使った実装例
~~~html
{% load static %}
<img src="{% static 'shop/images/no-image.png' %}">
<img src="{% static 'images/logo.png' %}"
~~~

#### `static`テンプレートタグ
- 設定済みの`STATICFILES_STORAGE`を含む相対パスのURLを生成する

### より詳しい説明
公式ドキュメントの[静的ファイル (画像、JavaScript、CSS など) を管理する](https://docs.djangoproject.com/ja/2.2/howto/static-files/)を参照。

## 10.5 メディアファイル関連の設定
### メディアファイル
- 静的ファイルのうち、ユーザーがサイトを利用してアップロードするCSVやPDF、画像などのファイル

#### 本番環境用のメディアファイル関連の設定例
~~~python
MEDIA_URL = '/media/'
MEDIA_ROOT = '/var/www/{}/media'.format(PROJECT_NAME)
~~~

#### テスト環境やHerokuなどのPaaSにDjangoをデプロイする際に「[WhiteNoise](http://whitenoise.evans.io/en/stable/)」を利用して静的ファイルを配信する場合の設定例
~~~python
MEDIA_ROOT = os.path.join(BASE_DIR, 'media_root')
~~~

### メディアファイルのアップロード
- `FileField`や`ImageField`を持つモデルを`save()`する
- `FileField`や`ImageField`を持つモデルをフォームとした画面でフィールドにファイルをアップロードする

#### `Form`を使った画面からのファイルアップロード実装時の注意点
1. `<form>`要素に`enctype="multipart/form-data"`属性を追加する
1. `ModelForm`をインスタンス化する際の第2引数に`request.FILES`を指定する
1. `FileField`や`ImageField`の`upload_to`オプションを使うことで、アップロード先のディレクトリやファイル名を動的に変更できる
1. Djangoサイトを可動させているサーバを冗長構成にしている場合は、S3等のストレージサービスやNFSを利用する

### アップロードしたメディアファイルの配信用URLを取得する
モデルの`FileField`や`ImageField`のフィールドから、アップロードしたメディアファイルの配信用URLを取得することができる

#### テンプレートから配信用URLを取得して画像を表示する実装例
~~~html
{% load static %}
...（略）...
{% if book.image %}
    <img src="{{ book,image.url }}">
{% else %}
    <img src="{% static 'shop/images/no-image.png' %}">
{% endif %}
~~~

### 開発時にメディアファイルの動作確認を行うための設定
#### `config/urls.py`
~~~python
from django.conf import settings
from django.conf.urls.static import static

urlpatterns = [
    ...
]
urlpatterns += static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT)
~~~

## 10.6 データベースの設定
### `DATABASES`
- データベースへのコネクションやその他オプションに関する設定を記述する

#### 組み込みでサポートしているデータベースエンジン
- `django,db.backends.postgresql`（PostgreSQL）
- `django.db.backends.mysql`（MySQL）
- `django.db.backends.sqlite3`（SQLite）
- `django.db.backends.oracle`（Oracle）

#### 組み込みでサポートしていないデータベースエンジンについて
サードパーティ製のライブラリをインストールすることで対応が可能

### 設定項目

| 項目 | 説明 | 設定例 |
| :--- | :--- | :----- |
| ENGINE | データベースエンジン | 'django.db.backends.mysql' |
| NAME | データベース名 | 'mysite' |
| USER | 接続するユーザー名 | 'mysiteuser' |
| PASSWORD | 接続するユーザーのパスワード | 'mysiteuserpass' |
| HOST | 接続先ホスト。省略した場合は'localhost' | 'localhost' |
| PORT | 接続先ポート。文字列か数値で指定する。<br />省略した場合はデフォルトのポート番号が使われる。 | '3306' |
| ATOMIC_REQUESTS | トランザクションの有効範囲を<br />リクエストの開始から終了までにするかどうかを指定 | True（デフォルトはFalse） |
| OPTIONS | 各種オプション | （後述） |

- データベースがトランザクションをサポートしている場合は、トランザクションの有効範囲についての設定を追加することができる
- `OPTIONS`には様々なオプション設定をすることが出来る
  - `isolation_level`：トランザクションの分離レベル
  - `sql_mode`：MySQLの場合のSQLモード

### MySQLの場合
####  最小設定例
~~~python
DATABASES = {
    'default': {
        'ENGINE': 'django.db,backend.mysql',
        'NAME': 'mysite',
        'USER': 'mysiteuser',
        'PASSWORD': 'mysiteuserpass',
    }
}
~~~

#### 設定例
~~~python
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.mysql',
        'NAME': 'mysite',
        'USER': 'mysiteuser',
        'PASSWORD': 'mysiteuserpass',
        'HOST': 'localhost',
        'PORT': '3306',
        'ATOMIC_REQUESTS': True,
        'OPTIONS': {
            'sql_mode': 'TRADITIONAL,NO_AUTO_VALUE_ON_ZERO',
        }
    }
}
~~~